In [ ]:
!pip install requests beautifulsoup4
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=4677d280a3bfcf3ccd8416e88c95ece160666fcaccbae8842fbbd88dbfaeab6e
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import data_table
import matplotlib.pyplot as plt
from urllib.request import urlopen
import re

## Scraping Data

In [ ]:
soup = BeautifulSoup(requests.get('http://www.spotrac.com/mlb/').text, 'html.parser')
teams = []
links = soup.find_all('a')
for link in links:
    url = link.get('href')
    if 'mlb' in url and '//payroll' in url:
      payroll_link = link.get('href')
      teams.append(re.match('https://www.spotrac.com/mlb/(.*)//payroll', payroll_link)[1])
print(teams)

['arizona-diamondbacks', 'atlanta-braves', 'baltimore-orioles', 'boston-red-sox', 'chicago-cubs', 'chicago-white-sox', 'cincinnati-reds', 'cleveland-guardians', 'colorado-rockies', 'detroit-tigers', 'houston-astros', 'kansas-city-royals', 'los-angeles-angels', 'los-angeles-dodgers', 'miami-marlins', 'milwaukee-brewers', 'minnesota-twins', 'new-york-mets', 'new-york-yankees', 'oakland-athletics', 'philadelphia-phillies', 'pittsburgh-pirates', 'san-diego-padres', 'san-francisco-giants', 'seattle-mariners', 'st-louis-cardinals', 'tampa-bay-rays', 'texas-rangers', 'toronto-blue-jays', 'washington-nationals']


In [ ]:
from tqdm import tqdm

def get_player_country(url):
  soup = BeautifulSoup(requests.get(url).text, 'html.parser')
  div = soup.find('div', attrs={'class': 'player-details'})
  for item in div.find_all('span', attrs={'class': 'player-item'}):
    if item.find('strong') and 'Country' in item.find('strong').get_text():
      return item.find('span', attrs={'class': 'player-infoitem'}).get_text().strip()
  return ""

def get_player_country_col(rows):
  countries = []
  for row in rows:
    for td in row.find_all('td'):
      links = td.find_all('a')
      if len(links) > 0:
        countries.append(get_player_country(links[0]['href']))
  return countries

def get_team_name(orig_team_name):
  return ' '.join([word.capitalize() for word in orig_team_name.split('-')])

def get_payroll_table(team, year):
  session = requests.Session()
  payload = {
      'email': 'sejaldua@gmail.com',
      'password': "@pple@pple"
  }

  login_url = 'https://www.spotrac.com/signin/submit/'
  session.post(login_url, payload)
  link = f'http://www.spotrac.com/mlb/{team}/payroll/{year}/'
  soup = BeautifulSoup(requests.get(link, cookies=session.cookies.get_dict()).text, 'html.parser')
  table = soup.find_all('table') 
  data = table[0]
  header = data.find_all("thead")[0]
  header_row = header.find_all('tr')
  header_data = [[td.get_text().strip() for td in header_row[i].find_all('th')] for i in range(len(header_row))]
  content = data.find_all("tbody")[0]
  rows = content.find_all('tr')
  rows_data = [[td.get_text().strip() for td in rows[i].find_all('td')] for i in range(len(rows))]
  df = pd.DataFrame(rows_data, columns=header_data)
  df.rename(columns={str(df.columns[0][0]): 'Active Players'}, inplace=True)
  df['Active Players'] = df.iloc[:, 0].apply(lambda x: x.split('\n')[1])
  df['Country'] = get_player_country_col(rows)
  df.columns = map(lambda x: x[0], list(df.columns))
  team_name = get_team_name(team)
  df.insert(0, "Team", [team_name]*len(df))
  # df = df.reset_index(drop=True).rename_axis(None, axis=1)
  return df[['Team', 'Active Players', 'Age', 'Pos.', 'Status',
      'Base Salary', 'Payroll %', 'Country']]

def get_concatenated_payroll_table(teams, year):
  df = pd.DataFrame()
  for team in tqdm(teams):
    df = pd.concat([df, get_payroll_table(team, year)], axis=0)
  return df

In [ ]:
df2019 = get_concatenated_payroll_table(teams, 2019)
df2019.shape

100%|██████████| 30/30 [30:45<00:00, 61.52s/it]


(1057, 8)

In [ ]:
# for team in teams:
#   for year in [2019, 2020, 2021, 2022]:
#     df = get_payroll_table(team, year)
#     df.to_csv(f'{team}-{year}.csv', index=False)

In [ ]:
df2019.to_csv('salary-data-2019.csv', index=False)

## Analysis

In [ ]:
!git clone https://github.com/sejaldua/mlb-racial-bias.git

Cloning into 'mlb-racial-bias'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [ ]:
!cd /content/mlb-racial-bias && ls

README.md	      salary-data-2018.csv  salary-data-2021.csv
salary-data-2016.csv  salary-data-2019.csv  salary-data-2022.csv
salary-data-2017.csv  salary-data-2020.csv


In [ ]:
import glob
import pandas as pd
# import pandasql as psql

df = pd.DataFrame()
for datafile in list(glob.glob("/content/mlb-racial-bias/*.csv")):
  temp_df = pd.read_csv(datafile)
  temp_df.rename(columns={'Active Players': 'Player', 'Pos.': 'Position', 'Base Salary': 
                          'Salary', 'Payroll %': 'Percentage'}, inplace=True)
  temp_df.insert(0, "Year", [datafile.strip('.csv')[-4:]]*len(temp_df))
  df = pd.concat([df, temp_df])
df.shape

(6781, 9)

In [ ]:
df['Year'] = df['Year'].astype(int)
df = df.sort_values('Year')
df.to_csv('./mlb-payroll-data-all-seasons.csv', index=False)
df.head()

,Year,Team,Player,Age,Position,Status,Salary,Percentage,Country
439,2016,Los Angeles Angels,Deolis Guerra,27,RP,Pre-Arb,"$507,500",0.21,Venezuela
361,2016,Houston Astros,Collin McHugh,29,SP,Pre-Arb,"$529,000",0.51,NaN
360,2016,Houston Astros,Yulieski Gurriel,32,1B,Pre-Arb,"$1,500,000",1.84,Cuba
359,2016,Houston Astros,Marwin Gonzalez,27,LF,Arb 2,"$2,000,000",1.94,Venezuela
358,2016,Houston Astros,Evan Gattis,29,C,Arb 1,"$3,300,000",3.19,NaN
